In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
sys.path.append(os.path.abspath('../modules'))
from utils import load_phishing_dataset
from preproc import PhishingDatasetPreproc